### 메일 보내기 (Test, 1명 대상)

In [5]:
import smtplib
from os.path import basename
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pandas as pd

# 개인 메일 정보 불러오기
df = pd.read_csv('Master/Mail_Private.csv')
id = df['id'][0]
pw = df['pw'][0]

# 보낼 Test file 정보
file = r'C:\Data\Sample\8_Pay\Result2\pw_1001잔머리 대마왕.pdf'

# 받을 Test 메일 주소
toid = id

# 세션 생성
s = smtplib.SMTP('smtp.gmail.com', 587)

# TLS 보안 시작
s.starttls()

# 로그인 인증
s.login(id,pw)

# 보낼 메시지 설정
msg = MIMEMultipart()
msg['Subject'] = '제목 : 메일 보내기 테스트입니다.'
msg['From'] = id
msg['To'] = toid
msg.attach(MIMEText('내용 : 본문내용 테스트입니다.'))

# 첨부 파일을 보내기 위한 사전 작업
fil = open(file, "rb")

# 첨부 파일을 메일에 붙이는 작업
part = MIMEApplication(fil.read(),Name=basename(file))

# 첨부된 파일에 대한 정보 추가
part['Content-Disposition'] = 'attachment; filename="%s"' % basename(file)
msg.attach(part)


# 메일 보내기
s.sendmail(id, toid, msg.as_string())

# 세션 종료
s.quit()


(221,
 b'2.0.0 closing connection j24-20020a170902759800b001c5f0fe64c2sm1975592pll.56 - gsmtp')

### 개별 급여 명세표 생성후 자동 메일링

In [1]:
import pandas as pd
import win32com.client
from PyPDF2 import PdfReader, PdfWriter
import os
import smtplib
from os.path import basename
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# 개인 메일 정보 불러오기
dfmail = pd.read_csv('Master/Mail_Private.csv')
id = dfmail['id'][0]
pw = dfmail['pw'][0]

# 메일 서비스를 위한 준비
# 세션 생성
s = smtplib.SMTP('smtp.gmail.com', 587)

# TLS 보안 시작
s.starttls()

# 로그인 인증
s.login(id,pw)

# 파일 경로 지정하기 TPath : 취합급여명세표 파일,
#                  RPath : PDF 결과 저장 폴더, 
#                  pwPath : 암호 설정된 PDF 저장 폴더
TPath = r'C:\Data\Sample\8_Pay\Data\취합급여명세표.xlsx'
RPath = r'C:\Data\Sample\8_Pay\Result'
pwPath = r'C:\Data\Sample\8_Pay\Result2'

# 각각의 시트의 값 읽어오기 df : Raw, dfM : Master, dfP : Private
df = pd.read_excel(TPath, sheet_name = 'Raw')
dfM = pd.read_excel(TPath, sheet_name = 'Master')
dfP = pd.read_excel(TPath, sheet_name = 'Private')

# 사번 정보 기준으로 df와 dfP 데이터 결합하기
df = df.merge(dfP, on='사번')

# None 값 0으로 대체하기
df = df.fillna(0)

#엑셀 실행하고 Format 시트를 ws로 선언하기
excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = True
wb = excel.Workbooks.Open(TPath)
ws = wb.Sheets("Format")

#각 인원에 대한 정보 반복문
for index, row in df.iterrows():
    #각 인원에 대한 사번, 이름 정보 가져오기
    ws.Cells(2,2).Value = row['사번']
    ws.Cells(2,4).Value = row['이름_x']

    #지급 공제 입력줄 초기화
    pnum = 5
    cnum = 5

    # 칼럼 명에 대한 반복
    for cName in df:
        # 구분자 생성    
        Sep = dfM['구분'].where(dfM['항목'] == cName).dropna()
        # 구분자가 값이 없을때
        if Sep.empty:
            if cName == '비고':
                ws.Cells(26,1).Value = row[cName]
        # 구분이 지급일 경우
        elif Sep.iloc[0] =="지급" and row[cName] > 0 :
            ws.Cells(pnum,1).Value = cName
            ws.Cells(pnum,2).Value = row[cName]
            pnum += 1
        # 구분이 공제일 경우
        elif Sep.iloc[0] =="공제" and row[cName] > 0 :
            ws.Cells(cnum,3).Value = cName
            ws.Cells(cnum,4).Value = row[cName]
            cnum += 1

    # PDF 파일로 파일 저장
    FName = str(row['사번']) + str(row['이름_x']) + '.pdf'
    ws.ExportAsFixedFormat(0, os.path.join(RPath,FName) )

    # 지급/공제 값 초기화     
    ws.Range("A5:D20").Value = ''

    # 암호가 걸린 위해 PDF 파일명 불러오기
    input_pdf = PdfReader(open(os.path.join(RPath,FName), "rb"))

    # PDF 파일 저장을 위한 Writer 생성하고 패스워드 설정하여 내용 입력
    output_pdf = PdfWriter()
    output_pdf.append_pages_from_reader(input_pdf)
    output_pdf.encrypt(str(row['PW']))

    # 만들어진 결과 파일을 저장 ※ wb 옵션은 파일을 저장하기 위한 binary 포멧이라는 의미
    output_pdf.write(open(os.path.join(pwPath, 'pw_' + FName), "wb"))   

    # 메일 보내기
    # 보낼 메시지 설정
    msg = MIMEMultipart()
    msg['Subject'] = row['이름_x'] + '님의 급여 명세표 송부의 건'
    msg['From'] = id
    msg['To'] = row['e-mail']
    msg.attach(MIMEText(row['이름_x'] + '님의 급여 명세표를 유첨 파일과 같이 송부 드립니다.'))

    # 첨부 파일을 보내기 위한 사전 작업
    file = os.path.join(pwPath, 'pw_' + FName)
    fil = open(file, "rb")

    # 첨부 파일을 메일에 붙이는 작업
    part = MIMEApplication(fil.read(),Name=basename(file))

    # 첨부된 파일에 대한 정보 추가
    part['Content-Disposition'] = 'attachment; filename="%s"' % basename(file)
    msg.attach(part)


    # 메일 보내기
    s.sendmail(id, row['e-mail'], msg.as_string())

# 세션 종료
s.quit()

# 엑셀 닫고 종료
wb.Close(False)
excel.Quit()
